# **MeSH - Extract Most Frequent Syntactic Structures**

Extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessous).

Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

In [43]:
import pandas
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/'

with open(path + "mesh-fr.txt", 'r', encoding = "UTF-8") as f:
    mesh = f.read().split("\n")

### POS tagging
TreeTagger

In [2]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [38]:
path = '/Users/camilledemers/Documents/04-filtrage/mapping_treeTagger_lefff.csv'

with open(path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}
mapping

{'ABR': '-',
 'ADJ': 'adj',
 'ADV': 'adv',
 'DET:ART': 'det',
 'DET:POS': 'det',
 'INT': '-',
 'KON': 'csu',
 'NAM': 'np',
 'NOM': 'nc',
 'NUM': '-',
 'PRO': 'pro',
 'PRO:DEM': 'pro',
 'PRO:IND': 'pro',
 'PRO:PER': 'pro',
 'PRO:POS': 'pro',
 'PRO:REL': 'pro',
 'PRP': 'prep',
 'PRP:det': 'prep',
 'PUN': '-',
 'PUN:cit': '-',
 'SENT': '-',
 'SYM': '-',
 'VER:cond': 'v',
 'VER:futu': 'v',
 'VER:impe': 'v',
 'VER:impf': 'v',
 'VER:infi': 'v',
 'VER:pper': 'v',
 'VER:ppre': 'v',
 'VER:pres': 'v',
 'VER:simp': 'v',
 'VER:subi': 'v',
 'VER:subp': 'v'}

In [36]:
mesh_txt = [tagger.tag_text(term) for term in mesh]

In [46]:
"""
0 - Token
1 - POS
2 - Lemme
"""

structures = []
tuples = []
for terme in mesh_txt:
    exp = " ".join([(t.split("\t")[0]) for t in terme])
    chunk = " ".join([mapping[t.split('\t')[1]] for t in terme if not '-' in mapping[t.split('\t')[1]]])
    tuples.append((exp, chunk))
    structures.append(chunk)

In [48]:
tuples

[('A-23187', 'np'),
 ('Téméfos', 'nc'),
 ('Abattoirs', 'nc'),
 ('Abattoir', 'nc'),
 ('Abréviations comme sujet', 'nc adv adj'),
 ('Acronymes comme sujet', 'nc adv adj'),
 ('Abdomen', 'nc'),
 ('Ventre', 'nc'),
 ('Abdomen aigu', 'nc adj'),
 ('Abdomen chirurgical', 'nc adj'),
 ('Abdomen urgent', 'nc adj'),
 ("Traumatismes de l' abdomen", 'nc prep det nc'),
 ('Blessures abdominales', 'nc adj'),
 ("Blessures de l' abdomen", 'nc prep det nc'),
 ('Lésions abdominales', 'nc adj'),
 ("Lésions de l' abdomen", 'nc prep det nc'),
 ('Lésions traumatiques abdominales', 'nc adj adj'),
 ("Lésions traumatiques de l' abdomen", 'nc adj prep det nc'),
 ('Traumatisme abdominal', 'nc adj'),
 ('Traumatismes abdominaux', 'nc adj'),
 ("Tumeurs de l' abdomen", 'nc prep det nc'),
 ('Tumeurs abdominales', 'nc adj'),
 ('Muscles abdominaux', 'nc adj'),
 ('Muscle abdominal', 'nc adj'),
 ("Muscles de l' abdomen", 'nc prep det nc'),
 ('Sangle abdominale', 'nc adj'),
 ('Nerf abducens', 'nc adj'),
 ('Nerf crânien 6', 'n

### Frequency distribution
To CSV

In [49]:
from nltk.probability import FreqDist
freq = FreqDist(structures)
tab = pandas.DataFrame(list(freq.items()), columns= ["Structure", "Fréquence"])
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
tab.to_csv(path+'mesh_patterns-fr.csv')
tab


,Structure,Fréquence
3,nc adj,20354
1,nc,16153
0,np,10051
40,nc prep nc,9450
153,nc np,5032
...,...,...
2432,nc nc prep nc prep det adj nc,1
2430,np np np nc nc adj nc,1
2429,nc adj prep v det nc,1
2428,nc adj prep prep nc prep nc,1
